# Numba

Numba – это компилятор с открытым исходным кодом, использующий подход LLVM (Low Level Virtual Machine). Numba использует компиляцию JIT (Just-in-time) – это означает, что компиляция выполняется во время выполнения кода Python, а не раньше!

Как мы увидим далее, лучше всего numba раскрывается при работе с циклами, обрабатывающие массивы данных.

Устанавливается numba обычным способом: либо через
```
pip install numba
```
либо
```
conda install numba
```

Ссылка на документацию: https://numba.readthedocs.io/en/stable/index.html

## Модельный пример, когда numpy не помогает
Допустим, у нас есть очень большой массив, и нам нужно найти ее монотонно возрастающую версию, такую, что значения только увеличиваются. Пример:
```
[1, 2, 1, 3, 3, 5, 4, 6] -> [1, 2, 2, 3, 3, 5, 5, 6]
```

In [1]:
import numpy as np

In [5]:
def monotonically_increasing(a):
    max_value = 0
    for i in range(len(a)):
        if a[i] > max_value:
            max_value = a[i]
        a[i] = max_value

In [6]:
a = np.array([1, 2, 1, 3, 3, 5, 4, 6])
monotonically_increasing(a)
a

array([1, 2, 2, 3, 3, 5, 5, 6])

Хотя аргументом функции и является numpy array, однако его обработка происходит в обычном питоновском цикле, что плохо сказывается на скорости работы:

In [12]:
%%timeit
N = 10_000_000
a = np.random.randint(0, N, size=N)
monotonically_increasing(a)

778 ms ± 78 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Основной способ использовать numba - обернуть имеющуюся функцию в декоратор:
jit или njit

In [5]:
from numba import njit

In [15]:
@njit
def monotonically_increasing(a):
    max_value = 0
    for i in range(len(a)):
        if a[i] > max_value:
            max_value = a[i]
        a[i] = max_value

In [16]:
a = np.array([1, 2, 1, 3, 3, 5, 4, 6])
monotonically_increasing(a)
a

array([1, 2, 2, 3, 3, 5, 5, 6])

In [17]:
%%timeit
N = 10_000_000
a = np.random.randint(0, N, size=N)
monotonically_increasing(a)

83 ms ± 129 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


В первый раз, когда вызывается функция, декорированная njit, нужно сгенерировать необходимый машинный код. Это занимает некоторое время. 

In [21]:
@njit
def add(a, b): 
    return a + b

In [22]:
%time add(1, 2)
%time add(1, 2)

CPU times: user 12.8 ms, sys: 3.08 ms, total: 15.9 ms
Wall time: 15.4 ms
CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.81 µs


3

при этом разные типы аргументов требуют генерации различных функций

In [23]:
%time add(1.5, 2.5)
%time add(1.5, 2.5)

CPU times: user 27.7 ms, sys: 0 ns, total: 27.7 ms
Wall time: 27.7 ms
CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 5.72 µs


4.0

При необходимости можно задать в декораторе аргументы функции. Тогда функция скомпилируется сразу. Это может быть полезно, например, при контролле точности чисел с плавующей точкой

In [6]:
from numba import jit, int32

@njit(int32(int32, int32))
def f(x, y):
    return x + y

In [44]:
%time f(1, 1)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 8.58 µs


2

# nopython
Numba может собирать код в двух режимах: *nopython mode* и *object mode*. При это вначале пытается быть реализована сборка в режиме nopython, а при неуспехе - в обьектном режиме. В случае обьектного режима, numba находит в коде части (в частности - циклы), которые можно реализовать в nopython моде, и выносит их в отдельные функции. 
При использовании @njit или @jit(nopython=True) переход в обьектный режим запрещается.

In [47]:
from numba import jit
import pandas as pd

x = {'a': [1, 2, 3], 'b': [20, 30, 40]}

@jit
def use_pandas(a): # Function will not benefit from Numba jit
    df = pd.DataFrame.from_dict(a) # Numba doesn't know about pd.DataFrame
    df += 1                        # Numba doesn't understand what this is
    return df.cov()                # or this!

print(use_pandas(x))

      a      b
a   1.0   10.0
b  10.0  100.0


/tmp/ipykernel_14899/2792547010.py:6: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
/tmp/ipykernel_14899/2792547010.py:6: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "use_pandas" failed type inference due to: non-precise type pyobject
During: typing of argument at /tmp/ipykernel_14899/2792547010.py (6)

File "../../../../../tmp/ipykernel_14899/2792547010.py", line 6:
<source missing, REPL/exec in use?>

  @jit
/home/nebil/.local/lib/python3.11/site-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "use_pandas" was compiled in object mode without forceobj=True.

File "../../../../../tmp/ipykerne

Любая строчка в коде выше не может быть интерпретирована numba, поэтому в итоге будет вызываться обычной питоновский код (с учётами на доп затратами на попытка скомпилировать его)

# numba и numpy
Почти все функции из numpy поддерживаются внутри numba. При это реализации из numpy скорее всего будут работать лучше, чем ваши реализации с помощью numba:

In [48]:
@jit(nopython=True)
def matmul(A, B):
    C = np.zeros((A.shape[0], B.shape[1]))
    
    for i in range(A.shape[0]):
        for j in range(B.shape[1]):
            for k in range(A.shape[1]):
                C[i, j] += A[i, k] * B[k, j]
                
    return C

In [52]:
A = np.random.randn(128, 128)
B = np.random.randn(128, 128)

np.linalg.norm(A @ B - matmul(A, B))

3.441256531276139e-13

In [53]:
%%timeit
C = matmul(A, B)

1.34 ms ± 11.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [54]:
%%timeit
C = A @ B

21.1 µs ± 791 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Также навешивание @jit на чисто numpy функции лишь замедлит их:

In [59]:
def just_sum(A):
    return np.sum(A, axis=1)

@jit('float64[:](float64[:, :])', nopython=True)
def numba_just_sum(A):
    return np.sum(A, axis=1)

In [60]:
A = np.random.randn(2048, 2048)

In [61]:
%%timeit
just_sum(A)

688 µs ± 8.69 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [62]:
%%timeit
numba_just_sum(A)

23.8 ms ± 117 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Так что если ваши операции можно реализовать используя лишь функции из numpy - доп использование numba скорее всего лишь замедлит вас

# objmode

Как говорилось выше, если сборка кода в nopython моде не получается, то numba делает сборку в object режиме. Однако ей в этом плане можно "помочь", обернув куски кода, требующие objmode c помощью одноименного окружения. Этот кусок кода будет интерпретироваться как функция с тем возвращаемым значением, что указан в аргументе objmode:
```
    with objmode(res='restype'):  # annotate return type
```

In [17]:
from scipy.linalg import solve_triangular
from numba import objmode

In [22]:
@jit
def solve_with_qr(A, b):
    Q, R = np.linalg.qr(A)
    x = Q.T @ b
    
    x = solve_triangular(R, x)
    
    return x

/tmp/ipykernel_17491/2187657385.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [25]:
A = np.random.randn(2048, 2048)
b = np.random.randn(2048)
x = solve_with_qr(A, b)
np.linalg.norm(A @ x - b)

6.526257239074654e-12

In [27]:
%%timeit
solve_with_qr(A, b)

417 ms ± 45.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
@jit(nopython=True)
def solve_with_qr(A, b):
    Q, R = np.linalg.qr(A)
    x = Q.T @ b
    
    with objmode(x='float64[:]'):
        x = solve_triangular(R, x)
    
    return x

In [29]:
A = np.random.randn(2048, 2048)
b = np.random.randn(2048)

In [30]:
x = solve_with_qr(A, b)

In [31]:
%%timeit
solve_with_qr(A, b)

325 ms ± 30.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


# fasmath
IEEE 754 — широко используемый стандарт IEEE, описывающий формат представления чисел с плавающей точкой. В некоторых задачах от некоторых правил стандарта можно отказаться в пользу ускорение кода. В numba это можно сделать с помощью аргумента декоратора **fastmath=True**

In [32]:
@njit(fastmath=False)
def do_sum(A):
    acc = 0.
    # without fastmath, this loop must accumulate in strict order
    for x in A:
        acc += np.sqrt(x)
    return acc

@njit(fastmath=True)
def do_sum_fast(A):
    acc = 0.
    # with fastmath, the reduction can be vectorized as floating point
    # reassociation is permitted.
    for x in A:
        acc += np.sqrt(x)
    return acc

In [33]:
A = np.linspace(0, 1, 10000)
print(do_sum(A), do_sum_fast(A))

6666.4979252009025 6666.497925200883


In [34]:
%timeit do_sum(A)
%timeit do_sum_fast(A)

15.4 µs ± 7.87 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
3.9 µs ± 7.97 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Над флагами оптимизации можно иметь чуть больше контроля, включая и выключая их отдельно.
Список флагом можно найти тут:
https://llvm.org/docs/LangRef.html#fast-math-flags

In [35]:
def add_assoc(x, y):
    return (x - y) + y

print(njit(fastmath=False)(add_assoc)(0, np.inf)) # nan
print(njit(fastmath=True) (add_assoc)(0, np.inf)) # 0.0
print(njit(fastmath={'reassoc', 'nsz'})(add_assoc)(0, np.inf)) # 0.0
print(njit(fastmath={'reassoc'})       (add_assoc)(0, np.inf)) # nan
print(njit(fastmath={'nsz'})           (add_assoc)(0, np.inf)) # nan

nan
0.0
0.0
nan
nan


# vectorize
декоратор **@vectorize**, позволяет описать функцию, работающую с одним элементом, а после применить её к массиву.
Для примера напишем функцию, находящие кол-во чисел меньших n, взаимопростых с n.

In [36]:
def gcd(x, y):
    while (y):
        x, y = y, x % y
    return x

def phi(n):
    cnt = 0
    for i in range(1, n):
        if gcd(i, n) == 1:
            cnt += 1
    return cnt

In [37]:
a = np.random.randint(low=1, high=10**4, size=1000)

In [43]:
%%timeit

res = [phi(n) for n in a]

120 ms ± 437 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
from numba import vectorize

In [40]:
@njit
def numba_gcd(x, y):
    while (y):
        x, y = y, x % y
    return x

@vectorize('int64(int64)', nopython=True, target='parallel')
def phi(n):
    cnt = 0
    for i in range(1, n):
        if numba_gcd(i, n) == 1:
            cnt += 1
    return cnt

target='parallel' означает, что функция будет испольняться на многоядерном процессоре. Другие возможные опции: 'cpu' и 'cuda'

In [42]:
%%timeit

res = phi(a)

9.58 ms ± 46.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


# jitclass
При необходимости, в numba можно реализовать свой класс, с которым можно будет работать в nopython режиме


In [52]:
import numba
from numba.experimental import jitclass

In [53]:
@jitclass
class Bag(object):
    value: numba.int32
    array: numba.float32[:]
    
    def __init__(self, value):
        self.value = value
        self.array = np.zeros(value, dtype=np.float32)

    @property
    def size(self):
        return self.array.size

    def increment(self, val):
        for i in range(self.size):
            self.array[i] += val
        return self.array

    @staticmethod
    def add(x, y):
        return x + y

In [54]:
A = Bag(5)

или другой способ:

In [55]:
spec = [
    ('value', numba.int32),
    ('array', numba.float32[:]),
]
@jitclass(spec)
class Bag(object):
    
    def __init__(self, value):
        self.value = value
        self.array = np.zeros(value, dtype=np.float32)

    @property
    def size(self):
        return self.array.size

    def increment(self, val):
        for i in range(self.size):
            self.array[i] += val
        return self.array

    @staticmethod
    def add(x, y):
        return x + y

In [56]:
A = Bag(5)

# cfunc
Некоторый библиотеки (например, scipy) могут работать с функциями, скомпилированнами на С, для увелечения их производительности. Такую функцию можно сделать с помощью numba, используя декоратор **@cfunc** с аргументами - сигнатурой функции.

In [57]:
from scipy.integrate import quad

In [58]:
def intergrand(t):
    return np.exp(-t) / t**2

In [59]:
%%timeit
quad(intergrand, 1, np.inf)

51 µs ± 455 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [61]:
from numba import cfunc

In [62]:
@cfunc('float64(float64)')
def intergrand(t):
    return np.exp(-t) / t**2

При использовании декоратора **@cfunc** создается callback-функция **f.ctypes**, а также **address**, позволяющая вызывать функцию в внешних C/C++ библиотеках

In [63]:
%%timeit

quad(intergrand, 1, np.inf)

57.9 µs ± 287 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [64]:
%%timeit

quad(intergrand.ctypes, 1, np.inf)

4.08 µs ± 85.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [69]:
f = intergrand.address
f # &f при использовании в коде на C

140213363339376

# stencil
**@stencil** - декоратор, схожий с vectorize, позволяющий определять функцию относительно её соседей в массиве (например свертка или численная производная)

In [74]:
x = np.linspace(0, 1, num=10**4, endpoint=False)
sinx = np.sin(x)
sinx

array([0.00000000e+00, 9.99999998e-05, 1.99999999e-04, ...,
       8.41308856e-01, 8.41362908e-01, 8.41416950e-01])

In [78]:
from numba import stencil

In [77]:
@stencil
def first_derivative(a):
    return (a[1] - a[-1]) / (2e-4)

In [76]:
first_derivative(sinx)

array([0.        , 0.99999999, 0.99999998, ..., 0.54055472, 0.54047059,
       0.        ])